1. INSTALL NECESSARY LIBRARIES

In [82]:
pip install langchain langchain_community discord.py sentence-transformers faiss-cpu pypdf llama-cpp-python python-dotenv nest_asyncio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


2. IMPORT NECESSARY LIBRARIES

In [ ]:
import os
import discord
from discord.ext import commands
import pickle
import nest_asyncio
import sentence_transformers
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms import HuggingFaceHub
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
import warnings
warnings.filterwarnings("ignore")

3. SET API KEYS AND DIRECTORIES

In [84]:
load_dotenv()
PDF_DIRECTORY = "algorithms_docs"
FAISS_INDEX_PATH = "faiss_index"
DISCORD_TOKEN = os.getenv("DISCORD_TOKEN")


4. LOAD PDFS WITH METADATA

In [85]:
def load_pdfs_metadata(directory_path):
    documents = []
    for file in os.listdir(directory_path):
        if file.endswith('.pdf'):
            pdf_path = os.path.join(directory_path, file)
            try:
                # Determine document type from filename or folder structure
                doc_type = "unknown"
                if "lecture" in file.lower():
                    doc_type = "lecture"
                elif "hw" in file.lower():
                    if "sol" in file.lower():
                        doc_type = "homework_solution"
                    else:
                        doc_type = "homework"
                elif "review" in file.lower():
                    doc_type = "exam"
                elif "midterm" in file.lower():
                    doc_type = "exam"
                elif "practice" in file.lower():
                    doc_type = "practice_problem"
                    
                # Load PDF
                loader = PyPDFLoader(pdf_path)
                docs = loader.load()
                    
                    # Add metadata to each page
                for doc in docs:
                    doc.metadata["source_type"] = doc_type
                    doc.metadata["filename"] = file
                    
                documents.extend(docs)
                print(f"Loaded: {file} as {doc_type}")
            except Exception as e:
                print(f"Error loading {file}: {e}")
    return documents

In [86]:
documents = load_pdfs_metadata(PDF_DIRECTORY)
print(f"Loaded {len(documents)} document pages in total")

Loaded: HW07_sol.pdf as homework_solution
Loaded: HW06_sol.pdf as homework_solution
Loaded: HW05.pdf as homework
Loaded: Lecture 02.pdf as lecture
Loaded: HW04_sol.pdf as homework_solution
Loaded: Lecture 03.pdf as lecture
Loaded: HW10.pdf as homework
Loaded: HW04.pdf as homework
Loaded: Lecture 0.pdf as lecture
Loaded: HW06.pdf as homework
Loaded: PracticeProblems4.pdf as practice_problem
Loaded: Lecture 01.pdf as lecture
Loaded: HW07.pdf as homework
Loaded: HW03.pdf as homework
Loaded: Lecture 10.pdf as lecture
Loaded: Lecture 04.pdf as lecture
Loaded: PracticeProblems1.pdf as practice_problem


parsing for Object Streams


Loaded: Lecture 05.pdf as lecture
Loaded: Lecture 11.pdf as lecture
Loaded: HW02.pdf as homework
Loaded: Lecture 07.pdf as lecture
Loaded: Lecture 13.pdf as lecture
Loaded: PracticeProblems2.pdf as practice_problem
Loaded: PracticeProblems3.pdf as practice_problem
Loaded: Lecture 12.pdf as lecture
Loaded: Lecture 06.pdf as lecture
Loaded: HW05_sol.pdf as homework_solution
Loaded: HW01.pdf as homework
Loaded: HW10_sol.pdf as homework_solution
Loaded: MidtermB.pdf as exam
Loaded: Lecture 08.pdf as lecture
Loaded: MidtermA.pdf as exam
Loaded: HW09_sol.pdf as homework_solution
Loaded: Lecture 09.pdf as lecture
Loaded: HW08_sol.pdf as homework_solution
Loaded: HW01_sol.pdf as homework_solution
Loaded: HW09.pdf as homework
Loaded: HW08.pdf as homework
Loaded: Final Review.pdf as exam
Loaded: HW0.pdf as homework
Loaded: Midterm Review.pdf as exam
Loaded: HW03_sol.pdf as homework_solution
Loaded: HW02_sol.pdf as homework_solution
Loaded 1014 document pages in total


In [87]:
doc_types = {}
for doc in documents:
    doc_type = doc.metadata.get("source_type", "unknown")
    doc_types[doc_type] = doc_types.get(doc_type, 0) + 1

print("Document types distribution:")
for doc_type, count in doc_types.items():
    print(f"- {doc_type}: {count} pages")

Document types distribution:
- homework_solution: 43 pages
- homework: 24 pages
- lecture: 719 pages
- practice_problem: 100 pages
- exam: 128 pages


In [88]:
if documents:
    print(f"Preview of the first document ({documents[0].metadata["filename"]}):")
    preview_text = documents[0].page_content[:500] + "..." if len(documents[0].page_content) > 500 else documents[0].page_content
    print(preview_text)

Preview of the first document (HW07_sol.pdf):
Johnson and Mirzaei CS 3330 Homework 7 Solution
Problem 1
(a) A = [2, 6, 7, 1, 3, 5, 4]
(b) j will never reach 6, as the loop is exclusive. It iterates through 0 to 5 for j.
Value ofj Value ofi Array
0 0 [2,6,7,1,3,5,4]
1 0 [2,6,7,1,3,5,4]
2 0 [2,6,7,1,3,5,4]
3 1 [2,1,7,6,3,5,4]
4 2 [2,1,3,6,7,5,4]
5 2 [2,1,3,6,7,5,4]
(c) A = [2, 1, 3, 4, 7, 5, 6]
Problem 2
(a) Taking either the smallest or the largest element of A as the pivot would lead to the worst partition.
In this case, at each iteration, ...


5. CREATE CHUNKS

In [89]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, separators=["\n\n", "\n", ".", " ", ""])
chunks = text_splitter.split_documents(documents)
print(f"Created {len(chunks)} chunks from {len(documents)} document pages")

Created 1560 chunks from 1014 document pages


In [90]:
if chunks:
    print(f"Example chunk (from{chunks[0].metadata["filename"]}):\n")
    preview_chunk = chunks[0].page_content[:300] + "..." if len(chunks[0].page_content) > 300 else chunks[0].page_content
    print(preview_chunk)
    print("\nChunk metadata:", chunks[0].metadata)

Example chunk (fromHW07_sol.pdf):

Johnson and Mirzaei CS 3330 Homework 7 Solution
Problem 1
(a) A = [2, 6, 7, 1, 3, 5, 4]
(b) j will never reach 6, as the loop is exclusive. It iterates through 0 to 5 for j.
Value ofj Value ofi Array
0 0 [2,6,7,1,3,5,4]
1 0 [2,6,7,1,3,5,4]
2 0 [2,6,7,1,3,5,4]
3 1 [2,1,7,6,3,5,4]
4 2 [2,1,3,6,7,5,4]
...

Chunk metadata: {'producer': 'pdfTeX-1.40.26', 'creator': 'TeX', 'creationdate': '2024-11-19T19:48:06+00:00', 'moddate': '2024-11-19T19:48:06+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0', 'trapped': '/False', 'source': 'algorithms_docs/HW07_sol.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1', 'source_type': 'homework_solution', 'filename': 'HW07_sol.pdf'}


6. CREATE VECTOR STORAGE

In [91]:
embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(documents=chunks, embedding = embeddings)
vectorstore.save_local(FAISS_INDEX_PATH)
print(f"Vector store created and saved to {FAISS_INDEX_PATH}")

Vector store created and saved to faiss_index


7. RETRIEVER FOR FILTERING

In [92]:
def metadata_filter(doc_type):
    def filter_func(doc):
        return doc.metadata.get("source_type") == doc_type
    return filter_func

def create_retriever(doc_type = None):
    vs = FAISS.load_local(FAISS_INDEX_PATH, embeddings, allow_dangerous_deserialization=True)
    if doc_type:
        base_retriever = vs.as_retriever(search_kwargs={"k": 10})
        def filtered_retriever(query):
                docs = base_retriever.invoke(query)
                filtered_docs = [doc for doc in docs if doc.metadata.get("source_type") == doc_type]
                return filtered_docs[:4]
        return filtered_retriever
    else:
        return vs.as_retriever(search_kwargs={"k": 4})


8. SET UP LANGUAGE MODEL

In [ ]:
def setup_llm():
    return HuggingFaceHub(
        repo_id="mistralai/Mistral-7B-Instruct-v0.2",  # You can change to another model
        model_kwargs={"temperature": 0.1, "max_new_tokens": 2048, "n_ctx": 4096, "verbose": True},
        huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_API_TOKEN")
    )

# Try to setup the LLM
try:
    llm = setup_llm()
    print("LLM setup completed")
except Exception as e:
    print(f"Error setting up LLM: {e}")

LLM setup completed


9. CREATE RAG CHAIN

In [ ]:
template = """\
You are an algorithms teaching assistant for a computer science class.
Answer the question based solely on the following context from class materials.
Do not include any extra text, background information, or commentary—only your final answer.

When answering:
1. Be thorough and explain concepts clearly.
2. Use examples to illustrate complex algorithms when appropriate.
3. Include time and space complexity analysis when relevant.
4. If the answer isn't fully contained in the context, say so rather than making up information.
5. Don't hallucinate.

Context:
{context}

Question:
{question}

Chat History: {chat_history}

Your final answer must start with "Answer:" and include nothing else.

Answer:"""

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

retriever = self.retriever | format_docs

def create_rag_chain(retriever_func):
    if callable(retriever_func) and not hasattr(retriever_func, 'invoke'):
        retriever_chain = RunnablePassthrough() | retriever_func | format_docs
    else:
        retriever_chain = retriever_func | format_docs
    
    chain = (
        {"context": retriever_chain, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    return chain

def get_rag_chain_for_thread(self, thread_id):
    # Create a new memory if this thread doesn't have one yet
    if thread_id not in self.thread_memories:
        self.thread_memories[thread_id] = ConversationBufferMemory(
            memory_key="chat_history", 
            return_messages=True
        )
    
    memory = self.thread_memories[thread_id]

rag_chain = create_rag_chain(retriever)


10. SMART DOCUMENT SELECTION

In [95]:
def smart_query(question):
    question_lower = question.lower()
    if any(term in question_lower for term in ["solution", "answer", "solved", "how to solve"]):
        print("Using primarily homework solutions for this query...")
        custom_retriever = create_retriever("homework_solution")
    elif any(term in question_lower for term in ["lecture", "class", "taught", "professor"]):
        print("Using primarily lecture materials for this query...")
        custom_retriever = create_retriever("lecture")
    elif any(term in question_lower for term in ["exam", "test", "quiz", "midterm", "final"]):
        print("Using primarily exam materials for this query...")
        custom_retriever = create_retriever("exam")
    elif any(term in question_lower for term in ["homework", "assignment", "problem set"]):
        print("Using primarily homework materials for this query...")
        custom_retriever = create_retriever("homework")
    else:
        print("Using all course materials for this query...")
        custom_retriever = create_retriever()
    temp_chain = create_rag_chain(custom_retriever)
    return temp_chain.invoke(question)

11. TEST RAG SYSTEM

In [96]:
test_question = "Explain how merge-sort works"
try:
    answer = smart_query(test_question)
    print(answer)
except Exception as e:
    print(f"Error: {e}")
    print("\nYou may need to download a language model or start text-generation-webui.")

Using all course materials for this query...
Human: You are an algorithms teaching assistant for a computer science class.
Answer the question based solely on the following context from class materials.
Do not include any extra text, background information, or commentary—only your final answer.

When answering:
1. Be thorough and explain concepts clearly.
2. Use examples to illustrate complex algorithms when appropriate.
3. Include time and space complexity analysis when relevant.
4. If the answer isn't fully contained in the context, say so rather than making up information.
5. Don't hallucinate.

Context:
Mergesort implementation
tnputTFFwistFLFofFn elementsFfromFaFtotallyForderedFuniverseTF
zutputTFF—heFnFelementsFinFascendingForderT
c
MERGE-SORT(L)

8
Divide and conquer: merge sort

7
Divide and conquer: merge sort

Mergesort
独”ecursivelyFsortFleftFhalfTF
独”ecursivelyFsortFrightFhalfTF
独xergeFtwoFhalvesFtoFmakeFsortedFwholeT
a
l r s t w x z ” … —
merge results
l w r z ” t — s x …
i

12. CREATE DISCORD BOT

In [ ]:
import re
class AlgorithmsBot(commands.Bot):
    def __init__(self):
        intents = discord.Intents.default()
        intents.message_content = True
        super().__init__(command_prefix="!", intents=intents)

    async def on_ready(self):
        print(f'{self.user} has connected to Discord!')
        print(f'Bot is in {len(self.guilds)} servers')

    async def setup_hook(self):
        await self.add_cog(AlgorithmsCog(self))

class AlgorithmsCog(commands.Cog):
    def __init__(self, bot):  # Accept bot parameter
        self.bot = bot
        self.retriever = create_retriever()
        # Create the RAG chain
        self.rag_chain = create_rag_chain(self.retriever)
        self.thread_memories = {}

    @commands.command(name='algo')
    async def algo_command(self, ctx, *, question):

        thread = await ctx.message.create_thread(
            name = f"Question: {question[:50]}...",
            auto_archive_duration = 60
        )

        async with thread.typing():
            try:
                # Invoke the chain to get the raw output.
                raw_output = self.rag_chain.invoke(question)
                
                # Post-process the output to only include content after "Answer:"
                if "Answer:" in raw_output:
                    filtered_answer = raw_output.split("Answer:")[-1].strip()
                else:
                    filtered_answer = raw_output.strip()
                
                # Build the final message displaying just the question and answer
                response = f"**Question:** {question}\n\n**Answer:** {filtered_answer}"
                
                # Send the response in chunks if it is long
                if len(response) > 1900:
                    chunks = [response[i:i+1900] for i in range(0, len(response), 1900)]
                    for chunk in chunks:
                        await thread.send(chunk)
                else:
                    await thread.send(response)

                await thread.send("You can ask follow-up questions in this thread!")
            except Exception as e:
                await ctx.send(f"Error: {str(e)}")

    @commands.command(name='sources')
    async def source_filter_command(self, ctx, source_type, *, question):
        valid_sources = ["lecture", "homework", "homework_solution", "exam", "practice_problem", "all"]
        if source_type.lower() not in valid_sources:
            await ctx.send(f"Invalid source type. Use one of: {', '.join(valid_sources)}")
            return
        
        mapping = {
        "all": None,
        "solution": "homework_solution",
        "homework_solution": "homework_solution",
        "lecture": "lecture",
        "homework": "homework",
        "exam": "exam",
        "practice_problem": "practice_problem",
        }
        doc_type = mapping.get(source_type.lower())

        thread = await ctx.message.create_thread(
            name = f"Question: {question[:50]}...",
            auto_archive_duration = 60
        )
        
        async with thread.typing():
            try:
                custom_retriever = create_retriever(doc_type)
                temp_chain = create_rag_chain(custom_retriever)
                raw_output = temp_chain.invoke(question)

                if "Answer:" in raw_output:
                    filtered_answer = raw_output.split("Answer:")[-1].strip()
                else:
                    filtered_answer = raw_output.strip()
                
                source_display = "all sources" if source_type.lower() == "all" else f"{source_type} materials"
                response = f"**Question:** {question}\n**Source:** {source_display}\n\n**Answer:** {filtered_answer}"

                if len(answer) > 1850:
                    chunks = [answer[i:i+1850] for i in range(0, len(answer), 1850)]
                    for i, chunk in enumerate(chunks):
                        await thread.send(chunk)
                else:
                    await thread.send(response)
            except Exception as e:
                await thread.send(f"Error: {str(e)}")

    @commands.command(name='help_algo')
    async def help_command(self, ctx):      
        help_text = (
            "**Algorithms Bot Commands:**\n\n"
            "`!algo [question]` - Ask any algorithms question\n"
            "`!sources [type] [question]` - Search only specific source types\n"
            "  - Valid types: lecture, homework, solution, exam, all\n"
            "`!help_algo` - Show this help message\n\n"
            "**Examples:**\n"
            "`!algo How does quicksort work?`\n"
            "`!sources lecture What is dynamic programming?`"
        )
        await ctx.send(help_text)

    @commands.Cog.listener()
    async def on_message(self, message):
        # Ignore messages from the bot itself
        if message.author == self.bot.user:
            return
            
        # Check if the message is in a thread
        if isinstance(message.channel, discord.Thread):
            # Check if this thread was created by the bot
            if message.channel.owner_id == self.bot.user.id:
                # Don't process commands (to avoid double processing)
                if not message.content.startswith(self.bot.command_prefix):
                    async with message.channel.typing():
                        try:
                            # Process the follow-up question
                            raw_output = self.rag_chain.invoke(message.content)
                            
                            # Post-process the output
                            if "Answer:" in raw_output:
                                filtered_answer = raw_output.split("Answer:")[-1].strip()
                            else:
                                filtered_answer = raw_output.strip()
                            
                            # Build the response
                            response = f"**Follow-up:** {message.content}\n\n**Answer:** {filtered_answer}"
                            
                            # Send the response in chunks if needed
                            if len(response) > 1900:
                                chunks = [response[i:i+1900] for i in range(0, len(response), 1900)]
                                for chunk in chunks:
                                    await message.channel.send(chunk)
                            else:
                                await message.channel.send(response)
                        except Exception as e:
                            await message.channel.send(f"Error processing follow-up: {str(e)}")


In [98]:
nest_asyncio.apply()
bot = AlgorithmsBot()
bot.run(DISCORD_TOKEN)

[2025-04-09 14:11:39] [INFO    ] discord.client: logging in using static token
[2025-04-09 14:11:39] [INFO    ] discord.client: logging in using static token
[2025-04-09 14:11:39] [INFO    ] discord.client: logging in using static token
[2025-04-09 14:11:39] [INFO    ] discord.client: logging in using static token
[2025-04-09 14:11:39] [INFO    ] discord.client: logging in using static token
[2025-04-09 14:11:39] [INFO    ] discord.client: logging in using static token
[2025-04-09 14:11:39] [INFO    ] discord.client: logging in using static token
[2025-04-09 14:11:39] [INFO    ] discord.client: logging in using static token
[2025-04-09 14:11:39] [INFO    ] discord.client: logging in using static token
[2025-04-09 14:11:39] [INFO    ] discord.client: logging in using static token
[2025-04-09 14:11:40] [INFO    ] discord.gateway: Shard ID None has connected to Gateway (Session ID: 35bacecb73bf485346768051ad1ddcfa).
[2025-04-09 14:11:40] [INFO    ] discord.gateway: Shard ID None has conne

Algorithms RAG Bot#0980 has connected to Discord!
Bot is in 1 servers
